In [62]:
import pandas as pd
import numpy as np
import datetime as dt
import random

import warnings
warnings.filterwarnings("ignore")

In [150]:
def interpolate(x_list: list, y_list: list, z: float):
    if x_list != sorted(x_list):
        raise ValueError('x_list must be sorted ASC')
    for index, element in enumerate(x_list):
        if z <= element:
            delta = (z - x_list[index - 1]) / (x_list[index] - x_list[index - 1])
            answer = y_list[index - 1] + (y_list[index] - y_list[index - 1]) * delta
            break
    return answer

In [179]:
swap_points = pd.read_csv('RUB swap points.csv')
swap_points.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
swap_points

,Term,SW POINTS
0,ON,131
1,1W,933
2,2W,2269
3,1M,4156
4,2M,8174
5,3M,12766
6,6M,25723
7,9M,39130
8,12M,52760
9,2Y,101653


In [180]:
usd_rates = pd.read_csv('USD rates.csv')
usd_rates.rename({'USD Rate MKT Data': 'USD Rate', 'Unnamed: 1': 'MKT Data'}, axis=1, inplace=True)
usd_rates

,USD Rate,MKT Data,"Conv, adj",StartDate,EndDate
0,Libor3m,0.001295,NaN,10/28/21,1/28/22
1,EDZ1,99.810000,0.00014,12/15/21,3/15/22
2,EDH2,99.785000,0.00073,3/16/22,6/16/22
3,EDM2,99.680000,0.00165,6/15/22,9/15/22
4,EDU2,99.520000,0.00287,9/21/22,12/21/22
5,EDZ2,99.295000,0.00451,12/21/22,3/21/23
6,EDH3,99.110000,0.00636,3/15/23,6/15/23
7,EDM3,98.920000,0.00839,6/21/23,9/21/23
8,EDU3,98.690000,0.01104,9/20/23,12/20/23
9,EDZ3,98.585000,0.01380,12/20/23,3/20/24


In [181]:
# функция для подсчёта времени в годах между датой и сегодняшним днём

def years(date): 
    return abs((pd.to_datetime(date) - datetime.today() + timedelta(days=1)).days)/360

# dummy функция из первой части задания

def dummy_df(dates):
    df = []
    for date in dates:
        df.append(1/(1+((1.01**(1+years(date))-1)*0.01*random.randint(90, 110)*years(date))))
    return df

In [182]:
# Первый массив, который нам дан, получим прибавляя своп пойнты из swap_points к текущей дате
# воспользуемся конвенцией 360 дней в году, 30 дней в месяце, 7 дней в неделе

from datetime import datetime, timedelta

# функция, которая переводит столбец Term в число дней
def days(x):
    date_after = None
    if "W" in x:
        date_after = datetime.today() + timedelta(days=7*int(x[:-1]))
    elif "M" in x:
        date_after = datetime.today() + timedelta(days=30*int(x[:-1]))
    elif "Y" in x:
        date_after = datetime.today() + timedelta(days=360*int(x[:-1]))
    elif "ON" in x:
        date_after = datetime.today() + timedelta(days=1)
    return (date_after - datetime.today()).days

days('2Y')

720

In [183]:
swap_points['date1'] = swap_points['Term'].apply(lambda x: dt.date.today() + timedelta(days=days(x)))
swap_points.drop(['Term'], axis=1, inplace=True)
swap_points

,SW POINTS,date1
0,131,2021-11-14
1,933,2021-11-20
2,2269,2021-11-27
3,4156,2021-12-13
4,8174,2022-01-12
5,12766,2022-02-11
6,25723,2022-05-12
7,39130,2022-08-10
8,52760,2022-11-08
9,101653,2023-11-03


In [184]:
# Второй массив дат получим сдвигом дат первого на случайное число дней от 30 до 180
swap_points['date2'] = swap_points['date1'] + timedelta(days=random.randint(30, 180))
swap_points

,SW POINTS,date1,date2
0,131,2021-11-14,2021-12-17
1,933,2021-11-20,2021-12-23
2,2269,2021-11-27,2021-12-30
3,4156,2021-12-13,2022-01-15
4,8174,2022-01-12,2022-02-14
5,12766,2022-02-11,2022-03-16
6,25723,2022-05-12,2022-06-14
7,39130,2022-08-10,2022-09-12
8,52760,2022-11-08,2022-12-11
9,101653,2023-11-03,2023-12-06


In [185]:
# для массивов дат date1 и date2 с помощью dummy функции получаем дисконт факторы для доллара

swap_points['DF_USD_1'] = dummy_df(swap_points['date1'])
swap_points['DF_USD_2'] = dummy_df(swap_points['date2'])
swap_points

,SW POINTS,date1,date2,DF_USD_1,DF_USD_2
0,131,2021-11-14,2021-12-17,0.999975,0.998884
1,933,2021-11-20,2021-12-23,0.999790,0.998742
2,2269,2021-11-27,2021-12-30,0.999600,0.998584
3,4156,2021-12-13,2022-01-15,0.999170,0.997864
4,8174,2022-01-12,2022-02-14,0.998155,0.996529
5,12766,2022-02-11,2022-03-16,0.997192,0.995384
6,25723,2022-05-12,2022-06-14,0.991871,0.990551
7,39130,2022-08-10,2022-09-12,0.986356,0.983919
8,52760,2022-11-08,2022-12-11,0.981456,0.976023
9,101653,2023-11-03,2023-12-06,0.943939,0.934079


In [186]:
# считаем долларовые ставки для date1 и date2

date1 = swap_points['date1'].apply(lambda x: years(x))
date2 = swap_points['date2'].apply(lambda x: years(x))

swap_points['rate_USD_1'] = (1/swap_points['DF_USD_1'] - 1)/date1
swap_points['rate_USD_2'] = (1/swap_points['DF_USD_2'] - 1)/date2
swap_points

,SW POINTS,date1,date2,DF_USD_1,DF_USD_2,rate_USD_1,rate_USD_2
0,131,2021-11-14,2021-12-17,0.999975,0.998884,0.009125,0.011826
1,933,2021-11-20,2021-12-23,0.999790,0.998742,0.010807,0.011340
2,2269,2021-11-27,2021-12-30,0.999600,0.998584,0.010287,0.010860
3,4156,2021-12-13,2022-01-15,0.999170,0.997864,0.009971,0.012231
4,8174,2022-01-12,2022-02-14,0.998155,0.996529,0.011093,0.013482
5,12766,2022-02-11,2022-03-16,0.997192,0.995384,0.011264,0.013574
6,25723,2022-05-12,2022-06-14,0.991871,0.990551,0.016391,0.016123
7,39130,2022-08-10,2022-09-12,0.986356,0.983919,0.018444,0.019419
8,52760,2022-11-08,2022-12-11,0.981456,0.976023,0.018894,0.022503
9,101653,2023-11-03,2023-12-06,0.943939,0.934079,0.029695,0.033740


In [187]:
# считаем рублёвые ставки для date1 и из них рублёвые дисконт факторы для date1

swap_points['rate_RUB_1'] = swap_points['rate_USD_1'] + 0.000001*swap_points['SW POINTS']
swap_points['DF_RUB_1'] = 1/(1 + swap_points['rate_RUB_1']*date1)
swap_points

,SW POINTS,date1,date2,DF_USD_1,DF_USD_2,rate_USD_1,rate_USD_2,rate_RUB_1,DF_RUB_1
0,131,2021-11-14,2021-12-17,0.999975,0.998884,0.009125,0.011826,0.009256,0.999974
1,933,2021-11-20,2021-12-23,0.999790,0.998742,0.010807,0.011340,0.011740,0.999772
2,2269,2021-11-27,2021-12-30,0.999600,0.998584,0.010287,0.010860,0.012556,0.999512
3,4156,2021-12-13,2022-01-15,0.999170,0.997864,0.009971,0.012231,0.014127,0.998824
4,8174,2022-01-12,2022-02-14,0.998155,0.996529,0.011093,0.013482,0.019267,0.996799
5,12766,2022-02-11,2022-03-16,0.997192,0.995384,0.011264,0.013574,0.024030,0.994028
6,25723,2022-05-12,2022-06-14,0.991871,0.990551,0.016391,0.016123,0.042114,0.979377
7,39130,2022-08-10,2022-09-12,0.986356,0.983919,0.018444,0.019419,0.057574,0.958607
8,52760,2022-11-08,2022-12-11,0.981456,0.976023,0.018894,0.022503,0.071654,0.933137
9,101653,2023-11-03,2023-12-06,0.943939,0.934079,0.029695,0.033740,0.131348,0.791956


In [199]:
# интерполируем рублёвые дисконт-факторы от date1 на date2

for i in range(len(swap_points['DF_RUB_1'])-1):
    swap_points['DF_RUB_2'][i] = interpolate(list(date1), list(swap_points['DF_RUB_1']), years(swap_points['date2'][i]))
swap_points['DF_RUB_2'][12] = float('nan')
swap_points

,SW POINTS,date1,date2,DF_USD_1,DF_USD_2,rate_USD_1,rate_USD_2,rate_RUB_1,DF_RUB_1,DF_RUB_2
0,131,2021-11-14,2021-12-17,0.999975,0.998884,0.009125,0.011826,0.009256,0.999974,0.998554
1,933,2021-11-20,2021-12-23,0.999790,0.998742,0.010807,0.011340,0.011740,0.999772,0.998149
2,2269,2021-11-27,2021-12-30,0.999600,0.998584,0.010287,0.010860,0.012556,0.999512,0.997677
3,4156,2021-12-13,2022-01-15,0.999170,0.997864,0.009971,0.012231,0.014127,0.998824,0.996522
4,8174,2022-01-12,2022-02-14,0.998155,0.996529,0.011093,0.013482,0.019267,0.996799,0.993540
5,12766,2022-02-11,2022-03-16,0.997192,0.995384,0.011264,0.013574,0.024030,0.994028,0.988656
6,25723,2022-05-12,2022-06-14,0.991871,0.990551,0.016391,0.016123,0.042114,0.979377,0.971762
7,39130,2022-08-10,2022-09-12,0.986356,0.983919,0.018444,0.019419,0.057574,0.958607,0.949268
8,52760,2022-11-08,2022-12-11,0.981456,0.976023,0.018894,0.022503,0.071654,0.933137,0.920195
9,101653,2023-11-03,2023-12-06,0.943939,0.934079,0.029695,0.033740,0.131348,0.791956,0.778109


In [200]:
# из полученных дисконт-факторов посчитаем рубдевые ставки для date 2
# посчитаем своп пойнты для date2

swap_points['rate_RUB_2'] = (1/swap_points['DF_RUB_2'] - 1)/date2
swap_points['SW POINTS 2'] = 1000000*(swap_points['rate_RUB_2'] - swap_points['rate_USD_2'])
swap_points

,SW POINTS,date1,date2,DF_USD_1,DF_USD_2,rate_USD_1,rate_USD_2,rate_RUB_1,DF_RUB_1,DF_RUB_2,rate_RUB_2,SW POINTS 2
0,131,2021-11-14,2021-12-17,0.999975,0.998884,0.009125,0.011826,0.009256,0.999974,0.998554,0.015331,3505.545438
1,933,2021-11-20,2021-12-23,0.999790,0.998742,0.010807,0.011340,0.011740,0.999772,0.998149,0.016688,5348.812057
2,2269,2021-11-27,2021-12-30,0.999600,0.998584,0.010287,0.010860,0.012556,0.999512,0.997677,0.017837,6976.752492
3,4156,2021-12-13,2022-01-15,0.999170,0.997864,0.009971,0.012231,0.014127,0.998824,0.996522,0.019943,7712.379552
4,8174,2022-01-12,2022-02-14,0.998155,0.996529,0.011093,0.013482,0.019267,0.996799,0.993540,0.025169,11687.527207
5,12766,2022-02-11,2022-03-16,0.997192,0.995384,0.011264,0.013574,0.024030,0.994028,0.988656,0.033582,20008.006600
6,25723,2022-05-12,2022-06-14,0.991871,0.990551,0.016391,0.016123,0.042114,0.979377,0.971762,0.049114,32990.539602
7,39130,2022-08-10,2022-09-12,0.986356,0.983919,0.018444,0.019419,0.057574,0.958607,0.949268,0.063497,44078.152308
8,52760,2022-11-08,2022-12-11,0.981456,0.976023,0.018894,0.022503,0.071654,0.933137,0.920195,0.079443,56940.328002
9,101653,2023-11-03,2023-12-06,0.943939,0.934079,0.029695,0.033740,0.131348,0.791956,0.778109,0.136335,102594.129062
